In [346]:
import numpy as np
import pandas as pd 
from pandas import Series,DataFrame
from datetime import datetime
import xgboost as xgb 
from sklearn.linear_model import Lasso, Ridge

In [347]:
#read data 
train_data = pd.read_csv('train.csv',skipinitialspace=True,parse_dates=['purchase_date','release_date'])
test_data = pd.read_csv('test.csv',skipinitialspace=True,parse_dates=['purchase_date','release_date'])
unit_data = pd.concat([train_data,test_data],axis=0) 
unit_data.reset_index(drop=True,inplace=True)
unit_data.to_csv('union_data.csv')

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [348]:
#data preprocess
positive_reviews_mean = unit_data['total_positive_reviews'].dropna().mean()
unit_data['total_positive_reviews'] = unit_data['total_positive_reviews'].fillna(positive_reviews_mean)
negative_reviews_mean = unit_data['total_negative_reviews'].dropna().mean()
unit_data['total_negative_reviews'] = unit_data['total_negative_reviews'].fillna(negative_reviews_mean)

In [349]:
#delete the noise
unit_data['purchase_date'] = unit_data['purchase_date'].fillna(unit_data['release_date'])
updated_train_data = unit_data.drop(index=76) 
updated_train_data = unit_data.drop(index=275)
updated_train_data = unit_data.drop(index=197)  
data_copy = updated_train_data

In [350]:
bins=[-1,5000,10000,15000,20000,1000000000] 
data_copy['tprice'] = pd.cut(data_copy['price'],bins,labels=['a','b','c','d','e'])


In [351]:
#from today
temp = pd.DatetimeIndex(data_copy['purchase_date'])
temp1=data_copy['purchase_date'].apply(lambda x:datetime.today()-x)
days = temp1.apply(lambda x:x.days)
#data_copy['days'] = days 

In [352]:
#scale
import sklearn.preprocessing as preprocessing
scaler = preprocessing.StandardScaler()
positive_param = scaler.fit(data_copy['total_positive_reviews'].values.reshape(-1,1))
data_copy['positive_scaled'] = scaler.fit_transform(data_copy['total_positive_reviews'].values.reshape(-1,1), positive_param)
negative_param = scaler.fit(data_copy['total_negative_reviews'].values.reshape(-1,1))
data_copy['negative_scaled'] = scaler.fit_transform(data_copy['total_negative_reviews'].values.reshape(-1,1), negative_param)
days_param = scaler.fit(days.values.reshape(-1,1)) 
data_copy['days_scaled'] = scaler.fit_transform(days.values.reshape(-1,1),days_param)

In [353]:
data_copy['rate'] = data_copy['total_positive_reviews']/(data_copy['total_positive_reviews']+data_copy['total_negative_reviews'])

In [354]:
from math import isnan
rate_mean = data_copy['rate'].dropna().mean()
for i,v in data_copy['rate'].items():
    if isnan(v):
        data_copy['rate'][i] = rate_mean
data_copy['rate'][65]


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


0.8487952904148063

In [355]:
#classify
dummies_genres = updated_train_data['genres'].str.get_dummies(sep=",").add_prefix('genres')
dummies_is_free = pd.get_dummies(data_copy['is_free'],prefix='is_free')
dummies_tprice = pd.get_dummies(data_copy['tprice'],prefix='tprice')
dummies_categories = updated_train_data['categories'].str.get_dummies(sep=",").add_prefix('categories') 
dummies_tags = updated_train_data['tags'].str.get_dummies(sep=",").add_prefix('tags') 

#print(dummies_genres)



In [356]:
#delete the not frequent class
for column in dummies_genres.columns:
    sum = 0
    for num in dummies_genres[column]:
        sum += num
    if sum<40:
        dummies_genres.drop(columns=column,axis =1,inplace = True)  

In [357]:
for column in dummies_categories.columns:
    sum = 0
    for num in dummies_categories[column]:
        sum += num
    if sum<40:
        dummies_categories.drop(columns=column,axis =1,inplace = True)

In [358]:
for column in dummies_tags.columns:
    sum = 0
    for num in dummies_tags[column]:
        sum += num
    if sum<40:
        dummies_tags.drop(columns=column,axis =1,inplace = True)

In [359]:
data_copy = pd.concat([data_copy,dummies_genres,dummies_is_free,dummies_tprice,dummies_categories,dummies_tags],axis=1) 

In [360]:
data_copy.drop(['is_free','price','genres','categories','tags','purchase_date','release_date','total_positive_reviews','total_negative_reviews','tprice'],axis=1,inplace=True)

In [361]:
data_copy.to_csv('whole_data1.csv',index=False)


In [362]:
data = data_copy
x_train = data.ix[0:356, 2:]  
y_train = data.ix[0:356,1:2] 
x_test = data.ix[357:,2:] 
x_train2 = train_data2.ix[0:, 2:]  
y_train2 = train_data2.ix[0:,1:2] 
x_test2 = data.ix[357:,2:]  
x_test 

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.
D:\Anaconda\lib\site-packages\ip

,positive_scaled,negative_scaled,days_scaled,rate,genresAction,genresAdventure,genresCasual,genresIndie,genresRPG,genresSimulation,...,tagsSingleplayer,tagsSpace,tagsStealth,tagsStory Rich,tagsStrategy,tagsSurvival,tagsThird Person,tagsTurn-Based,tagsVR,tagsViolent
357,-0.291113,-0.085355,-0.746930,0.699115,1,1,0,0,0,0,...,1,0,0,1,0,1,1,0,0,0
358,-0.213533,-0.035143,-1.599766,0.720520,1,1,0,0,0,0,...,1,1,0,1,1,0,0,1,0,0
359,-0.338324,-0.129972,-1.586893,0.837805,0,0,0,1,0,1,...,1,0,0,0,1,0,0,0,0,0
360,-0.353570,-0.134213,0.125216,0.632075,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
361,0.636819,0.031310,1.219421,0.915827,1,1,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
362,-0.345701,-0.132318,-0.273847,0.826923,0,1,0,1,1,0,...,1,0,0,1,0,0,0,0,0,0
363,-0.315186,-0.119731,-1.467817,0.818913,1,0,1,1,0,0,...,1,0,0,1,1,0,0,0,0,0
364,-0.348922,-0.132228,-0.038915,0.755162,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
365,-0.311620,-0.077776,-0.302811,0.578844,0,0,0,0,1,0,...,1,0,0,1,0,0,1,0,0,1
366,-0.260007,-0.090588,1.583083,0.793768,0,0,0,0,1,0,...,1,0,0,0,1,0,0,1,0,0


In [363]:
#xgoost
model = xgb.XGBRegressor(max_depth=9, learning_rate=0.1, n_estimators=200, silent=False, objective='reg:gamma')
model.fit(x_train, y_train) 
 

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=9, min_child_weight=1, missing=None, n_estimators=200,
             n_jobs=1, nthread=None, objective='reg:gamma', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=False, subsample=1, verbosity=1)

In [364]:

y_predict = model.predict(x_test)
#y_predict2 = model2.predict(x_test) 
#[rows,cols] = y_predict.shape
#for i in range(rows):
#    for j in range(cols):
#        if y_predict[i,j]< 0:
#            y_predict[i,j] = 0
y_predict

array([4.62117410e-12, 3.89205713e-27, 2.71746977e-25, 2.18184446e-11,
       2.88610768e+00, 4.77243939e-17, 1.11561412e-16, 7.82832978e-18,
       1.07609429e-14, 2.42908340e-04, 2.61451839e-20, 2.81701951e-29,
       4.75560725e-01, 3.71192917e-02, 5.26406366e-05, 3.27483122e-03,
       2.51039580e-08, 9.99999419e-03, 1.49443323e-07, 6.36633835e-04,
       1.36038776e-30, 5.40413022e-01, 3.74095544e-05, 1.57956015e-06,
       3.24790120e+00, 6.26836272e-09, 1.54769630e-30, 3.04707722e-03,
       1.62342942e-04, 2.32267263e-03, 7.33040622e-04, 2.96948338e+01,
       7.24599638e-04, 2.00862851e-12, 2.45473623e+00, 2.30601527e-05,
       1.06657095e-01, 4.37038326e+00, 2.24993342e-17, 2.55377337e-16,
       4.13734056e-02, 2.37717039e-20, 4.67769505e-21, 9.63303936e-28,
       5.49717472e-09, 1.53405502e-01, 6.09363653e-08, 1.20592233e-22,
       2.29683832e-26, 9.96935010e-01, 9.22291221e-08, 9.84420078e-26,
       1.77060819e-08, 1.07661519e-15, 9.94719177e-16, 1.04815437e-16,
      

In [365]:
prediction = pd.DataFrame(y_predict,columns=['playtime_forever'])
prediction.reset_index(drop = True,inplace = True)
prediction['id'] = range(len(prediction))
order = ['id','playtime_forever']
prediction = prediction[order]
prediction.to_csv('submission.csv',index = False)